In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, GRU, Dense, Embedding
from keras.optimizers import Adam
import matplotlib.pyplot as plt



In [14]:
url = 'data/eng_-french.csv'
data = pd.read_csv(url)

# Preprocessing
def preprocess_data(data):
    data.rename(columns={"English words/sentences":"Eng", "French words/sentences":"Frn"}, inplace=True)
    return data

data = preprocess_data(data)

# Tokenization and Padding
def tokenize_and_pad(texts, max_len=None):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)
    padded_sequences = pad_sequences(sequences, padding='post', maxlen=max_len)
    return padded_sequences, tokenizer

eng_sequences, eng_tokenizer = tokenize_and_pad(data['Eng'])
frn_sequences, frn_tokenizer = tokenize_and_pad(data['Frn'])

eng_max_len = max(len(seq) for seq in eng_sequences)
frn_max_len = max(len(seq) for seq in frn_sequences)

eng_sequences, eng_tokenizer = tokenize_and_pad(data['Eng'], max_len=eng_max_len)
frn_sequences, frn_tokenizer = tokenize_and_pad(data['Frn'], max_len=frn_max_len)

eng_vocab_size = len(eng_tokenizer.word_index) + 1
frn_vocab_size = len(frn_tokenizer.word_index) + 1

train_eng, test_eng, train_frn, test_frn = train_test_split(eng_sequences, frn_sequences, test_size=0.2, random_state=42)





In [15]:
# Define the Seq2Seq Model with GRU
def define_seq2seq_model(eng_vocab_size, frn_vocab_size, eng_max_len, frn_max_len, embedding_dim=256, gru_units=256):
    # Encoder
    encoder_inputs = Input(shape=(eng_max_len,))
    encoder_embedding = Embedding(eng_vocab_size, embedding_dim, input_length=eng_max_len, mask_zero=True)(encoder_inputs)
    encoder_gru = GRU(gru_units, return_state=True)
    _, state_h = encoder_gru(encoder_embedding)
    encoder_states = [state_h]

    # Decoder
    decoder_inputs = Input(shape=(frn_max_len,))
    decoder_embedding = Embedding(frn_vocab_size, embedding_dim, input_length=frn_max_len, mask_zero=True)(decoder_inputs)
    decoder_gru = GRU(gru_units, return_sequences=True, return_state=True)
    decoder_outputs, _ = decoder_gru(decoder_embedding, initial_state=encoder_states)
    decoder_dense = Dense(frn_vocab_size, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    # Define the model
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

    return model



In [ ]:
# Hyperparameters
embedding_dim = 256
gru_units = 256
batch_size = 64
epochs = 2
learning_rate = 1e-3

model = define_seq2seq_model(eng_vocab_size, frn_vocab_size, eng_max_len, frn_max_len, embedding_dim, gru_units)
model.compile(optimizer=Adam(learning_rate), loss='sparse_categorical_crossentropy')

# Training the model
train_frn_input = train_frn[:, :-1]  # input to the decoder
train_frn_output = train_frn[:, 1:]  # expected output of the decoder (shifted by one timestep)

# Padding train_frn_input to ensure it matches the required input shape
train_frn_input = pad_sequences(train_frn_input, maxlen=frn_max_len, padding='post')
train_frn_output = pad_sequences(train_frn_output, maxlen=frn_max_len, padding='post')

history = model.fit([train_eng, train_frn_input], train_frn_output,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.2)

# Plotting the loss
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training vs Validation Loss')
plt.show()



In [10]:
# Define the encoder model
encoder_model = Model(encoder_inputs, encoder_states)

# Define the decoder model
decoder_state_input_h = Input(shape=(gru_units,))
decoder_states_inputs = [decoder_state_input_h]

decoder_outputs, state_h = decoder_gru(
    decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)



English words/sentences    0
French words/sentences     0
dtype: int64

In [11]:
# Function to translate using the trained model
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1 with only the start character.
    target_seq = np.zeros((1, 1))
    
    # Start character is assumed to be index 1 (usually 0 is reserved for padding)
    target_seq[0, 0] = 1

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h = decoder_model.predict(
            [target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = frn_tokenizer.index_word.get(sampled_token_index, '')

        decoded_sentence += ' ' + sampled_word

        if (sampled_word == '' or
           len(decoded_sentence.split()) >= frn_max_len):
            stop_condition = True

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        states_value = [h]

    return decoded_sentence.strip()



In [12]:
for i in range(5):
    input_seq = test_eng[i:i+1]
    translated_sentence = decode_sequence(input_seq)
    print('Input:', data['Eng'].iloc[i])
    print('Translation:', translated_sentence)
    print()



--- Random English - French Sentence --- 
English Sentence/Word:  I know Tom has gone. 
French Sentence/Word:  Je sais que Tom a disparu.
